# data prepare

In [1]:
import pandas as pd
from PIL import Image
import torch

In [2]:
data = pd.read_csv('./data/list_attr_celeba.txt', sep="\s*", header=0)
partition = pd.read_csv('./data/list_eval_partition.txt', sep="\s*", header=None)

/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:2379: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/anaconda3/lib/python3.6/site-packages/pandas/io/parsers.py:2381: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [3]:
partition.columns = ["filename", "file_cate"]
data = data.merge(partition, on = "filename", how = "left")

In [4]:
print(data.columns)

Index(['filename', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young', 'file_cate'],
      dtype='object')


In [5]:
data.to_csv("./data/list_merge_attr_celeba.csv", index = False)

In [6]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')#将图片转换为RGB格式。

In [7]:
picture_data1=pil_loader(r'/home/tongqing.li/qianbao/picture_score/data/img_align_celeba/000001.jpg')

In [8]:
import numpy as np
np.array(picture_data1).shape

(218, 178, 3)

In [126]:
from torch.utils.data import Dataset, DataLoader 
import os
class myDataset(Dataset):
    def __init__(self, loadPath, transform, data_category = 0): #r'home/tongqing.li/lesson/pytorch/MNIST_data/MNIST/processed/training.pt'
        super(myDataset, self).__init__()
        self.data = pd.read_csv(loadPath)
        self.data = self.data[self.data["file_cate"] == data_category]
        self.transform = transform
        
    def __getitem__(self, index):
        #Heavy_Makeup，Male，Wearing_Earrings，Wearing_Necklace，Wearing_Necktie
        picture = pil_loader(r'/home/tongqing.li/qianbao/picture_score/data/img_align_celeba/' + self.data.iloc[index]["filename"])
        label = self.data.iloc[index][["Eyeglasses", "Heavy_Makeup", "Male", "Wearing_Earrings", "Wearing_Necklace", "Wearing_Necktie"]]
        for i in ["Eyeglasses", "Heavy_Makeup", "Male", "Wearing_Earrings", "Wearing_Necklace", "Wearing_Necktie"]:
            if (label.loc[i] == -1):
                label.loc[i] = 0
        label = torch.tensor(label)
        picture = self.transform(picture)
        return (picture, label.long())
    
    def __len__(self):
        return self.data.shape[0]

In [127]:
from torchvision import datasets, transforms
transform = transforms.Compose([transforms.Resize((224,224)), transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

train_face_dataset = myDataset("./data/list_merge_attr_celeba.csv", data_category = 0, transform = transform)
valid_face_dataset = myDataset("./data/list_merge_attr_celeba.csv", data_category = 1, transform = transform)
test_face_dataset = myDataset("./data/list_merge_attr_celeba.csv", data_category = 2, transform = transform)

In [131]:
train_face_dataset.__getitem__(0)[1].shape

torch.Size([6])

In [79]:
batch_size = 64
num_workers = 4

In [80]:
train_face_loader = DataLoader(train_face_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers, pin_memory=True)
valid_face_loader = DataLoader(valid_face_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers, pin_memory=True)
test_face_loader = DataLoader(test_face_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers, pin_memory=True)

In [150]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 16, 5)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 32, 3)
        # an affine operation: y = Wx + b 
        self.fc1 = nn.Linear(2048, 84)
        self.fc2 = nn.Linear(84, 2)
        self.fc3 = nn.Linear(84, 2)
        self.fc4 = nn.Linear(84, 2)
        self.fc5 = nn.Linear(84, 2)
        self.fc6 = nn.Linear(84, 2)
        self.fc7 = nn.Linear(84, 2)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool1(F.relu(self.conv2(x)))
        x = self.pool1(F.relu(self.conv3(x)))
        x = self.pool1(F.relu(self.conv3(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv3(x))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        y = []
        y.append(self.fc2(x))
        y.append(self.fc3(x))
        y.append(self.fc4(x))
        y.append(self.fc5(x))
        y.append(self.fc6(x))
        y.append(self.fc7(x))
        return y
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [226]:
from torch import nn, optim
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
#model = Net()

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)
epochs = 100
steps = 0

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for images, labels in train_face_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        
        log_ps = model(images)
        #print(labels.shape)
        loss = []
        for i, log_ps_data in enumerate(log_ps):
            loss.append(criterion(log_ps_data, labels[:,i]))
        #print(loss)
        sum(loss).backward()
        optimizer.step()
        
        running_loss += sum(loss).item()
    
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            model.eval()#这里就是关闭dropout了。
            for images, labels in valid_face_loader:
                images, labels = images.to(device), labels.to(device)
                log_ps = model(images)
                test_loss = []
                
                for i, log_ps_data in enumerate(log_ps):
                    test_loss.append(criterion(log_ps_data, labels[:,i]))
                    top_p, top_class = log_ps_data.topk(1, dim=1)
                    equals = top_class == labels[:, i].view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
        
        model.train()
        
        #model.load_state_dict(torch.load(filepath))
        torch.save(model.state_dict(), r'/home/tongqing.li/qianbao/deep_learning/neural_network_in_pytorch/deep_model/model' + str(e + 1))
        train_losses.append(running_loss/len(train_face_loader))
        test_losses.append(sum(test_loss)/len(valid_face_loader))
        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(train_losses[-1]),
              "Test Loss: {:.3f}.. ".format(test_losses[-1]),
              "Test Accuracy: {:.3f}".format(accuracy/len(test_face_loader)))

Epoch: 1/100..  Training Loss: 0.648..  Test Loss: 0.004..  Test Accuracy: 0.948


KeyboardInterrupt: 

In [228]:
        with torch.no_grad():    
            model.eval()#这里就是关闭dropout了。
            accuracy = [0, 0, 0, 0, 0, 0]
            for images, labels in valid_face_loader:
                images, labels = images.to(device), labels.to(device)
                log_ps = model(images)
                test_loss = []
                x = 0
                for i, log_ps_data in enumerate(log_ps):
                    test_loss.append(criterion(log_ps_data, labels[:,i]))
                    top_p, top_class = log_ps_data.topk(1, dim=1)
                    equals = top_class == labels[:, i].view(*top_class.shape)
                    accuracy[i] += torch.mean(equals.type(torch.FloatTensor))
                #print(accuracy)
            model.train()

In [229]:
tot = 0
for i in range(6):
    print(accuracy[i]/len(test_face_loader))
    tot = tot + accuracy[i]/len(test_face_loader)
print(tot/6)
#"Eyeglasses", "Heavy_Makeup", "Male", "Wearing_Earrings", "Wearing_Necklace", "Wearing_Necktie"

tensor(0.9881)
tensor(0.9012)
tensor(0.9691)
tensor(0.8914)
tensor(0.8621)
tensor(0.9532)
tensor(0.9275)


In [230]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg11_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg13', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg13_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg16_bn', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'vgg19_bn', pretrained=True)
model.eval()

Using cache found in /home/tongqing.li/.cache/torch/hub/pytorch_vision_v0.6.0
Downloading: "https://download.pytorch.org/models/vgg11-bbd30ac9.pth" to /home/tongqing.li/.cache/torch/checkpoints/vgg11-bbd30ac9.pth


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 